In [2]:
import os
import cv2
import torch
import numpy as np
import open3d as o3d

# Load MiDaS model for depth estimation
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large")
midas.to('cpu')
midas.eval()

# Load MiDaS transforms for input preprocessing
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

# Function to generate 3D point cloud from depth map
def generate_point_cloud_from_depth(depth_map, img_rgb):
    """Generates a 3D point cloud from a depth map."""
    height, width = depth_map.shape
    point_cloud = []
    colors = []

    # Camera intrinsic parameters (adjust as needed)
    fx = fy = 500  # Focal lengths
    cx, cy = width / 2, height / 2  # Principal point (image center)

    for v in range(height):
        for u in range(width):
            z = depth_map[v, u]
            if z > 0:  # Only consider valid depth values
                x = (u - cx) * z / fx
                y = (v - cy) * z / fy
                point_cloud.append([x, y, z])
                colors.append(img_rgb[v, u] / 255.0)  # Normalize color to [0, 1]

    return np.array(point_cloud), np.array(colors)

# Function to process each image and generate point cloud
def process_image(image_path, output_dir):
    # Load image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess image and convert to tensor
    input_batch = transform(img_rgb).to('cpu')

    # Perform depth estimation using MiDaS
    with torch.no_grad():
        prediction = midas(input_batch)

    # Get depth map as numpy array
    depth_map = prediction.squeeze().cpu().numpy()

    # Normalize depth map (optional)
    depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

    # Resize depth map to match the original image
    depth_map_resized = cv2.resize(depth_map, (img_rgb.shape[1], img_rgb.shape[0]))

    # Convert the depth map into a 3D point cloud
    point_cloud, colors = generate_point_cloud_from_depth(depth_map_resized, img_rgb)

    # Create Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # Generate output filename and save the point cloud
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path = os.path.join(output_dir, f"{image_name}.ply")
    o3d.io.write_point_cloud(output_path, pcd)

    print(f"3D point cloud saved to {output_path}")

# Main function to process all PNG files in a folder
def process_folder(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all PNG files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            process_image(image_path, output_folder)

# Specify the input folder containing PNG images and output folder
input_folder = r"C:\Users\Shashank\Desktop\Point Cloud\DATA\pothole-detection_JPGs\pothole"
output_folder = r"C:\Users\Shashank\Desktop\Point Cloud\DATA\output"

# Process the folder and generate point clouds for all images
process_folder(input_folder, output_folder)


Using cache found in C:\Users\Shashank/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Shashank/.cache\torch\hub\intel-isl_MiDaS_master


3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes0.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes1.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes10.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes100.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes101.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes102.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes103.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes104.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes105.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes106.ply
3D point cloud saved to C:\Users\Shashank\Desktop\Point Cloud\DATA\output\potholes107.ply
3D point cloud 